In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

def get_cor(d,y):
    regs=[]
    x=np.array(d)
    for i in range(len(x[0])):
        regs.append(stats.linregress(y,x[:,i]))
    rna=pd.DataFrame(np.array(regs)[:,2:4],index=d.columns,columns=['cor','p']).sort_values('cor',ascending=False)
    return rna

data='../data/'

In [2]:
%%time
clock_=data+'source/41467_2015_BFncomms9570_MOESM436_ESM.xlsx'
clock=pd.read_excel(clock_,skiprows=2,nrows=1497)
clock.index=clock['NEW-Gene-ID']
clock=clock[['RANK','META:Direction']]
clock=clock.rename(index={'GPR56':'ADGRG1'})
down=clock[clock['META:Direction']=='-']
up=clock[clock['META:Direction']=='+']

CPU times: user 639 ms, sys: 31.9 ms, total: 671 ms
Wall time: 956 ms


In [3]:
gen=pd.read_csv(data+'clean/genes.csv',index_col=0)
dic=gen.to_dict()['gen']
len(dic)

18937

In [4]:
%%time
rna=pd.read_pickle(data+'results/rna.pkl').drop('sex',axis=1)
cohorts=rna['cohort'].unique()
rna.shape

CPU times: user 385 ms, sys: 666 ms, total: 1.05 s
Wall time: 1.05 s


(3458, 18883)

In [5]:
genes=['ELOVL2','KLF14','FHL2','CD248','LRRN3','NELL2','SATB1']
top=rna[['cohort','sid','age']+[gen[gen['gen']==c].index[0] for c in genes]]
top.columns=['cohort','sid','age']+genes
top.to_csv(data+'results/top_rna.csv') # for Fig 2

In [6]:
%%time
rna=rna.drop('sid',axis=1)
rna_cors=[]
for cohort in cohorts:
    df=rna[rna['cohort']==cohort].drop('cohort',axis=1)
    dfm=np.log2(df.drop('age',axis=1)+1)
    dfm=(dfm.T-dfm.median(1)).T
    rna_cor=get_cor(dfm,df['age'])
    rna_cor['cohort']=cohort
    rna_cors.append(rna_cor)

CPU times: user 48.4 s, sys: 1.42 s, total: 49.8 s
Wall time: 49.9 s


In [7]:
pl=pd.concat(rna_cors).join(gen)
pl.loc[pl['gen'].isin(down.head(10).index),'top10']='down'
pl.loc[pl['gen'].isin(up.head(10).index),'top10']='up'
fig1b=pl[pl['top10'].isin(['up','down'])][['cor','cohort','top10','gen']]
fig1b.shape

(114, 4)

In [8]:
fig1b.to_csv(data+'figs/1b.csv')

In [9]:
pl.loc[pl['gen'].isin(up.index),'proposed']='up'
pl.loc[pl['gen'].isin(down.index),'proposed']='down'
pl.shape

(113280, 6)

In [10]:
fig1d=pl[['cohort','cor','proposed']].dropna()
fig1d.shape

(7962, 3)

In [11]:
fig1d.to_csv(data+'figs/1d.csv')

In [12]:
pl.to_csv(data+'results/rna_cor.csv')